## Random Forest para predecir compras ##
El principal objetivo de realizar Random Forest es buscar los features que aporten mayor información. Esto se debe a que random forest es un árbol de decisión con un ensamble, cada ensamble tiene features dintintos, combinandolos y utilizando muchos árboles buscaré la mayor precisión de clasificación y por lo tanto podré determinar que features son los más importantes.

In [1]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import xgboost as xgb
from sklearn import preprocessing

tf = pd.read_csv('../data/events_up_to_01062018.csv')
labels = pd.read_csv('../data/labels_training_set.csv')

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Divido las fechas**

In [2]:
tf["timestamp"] = pd.to_datetime(tf["timestamp"])
tf["month"] = tf["timestamp"].dt.month
tf["day"] = tf["timestamp"].dt.day
tf["hour"] = tf["timestamp"].dt.hour

In [3]:
labels.count() #cantidad de labels que tengo

person    19414
label     19414
dtype: int64

**Divido por evento me quedo con las columnas obligatorias por evento**

En en análisis exporatorio me había dado cuenta que no todos los eventos tienen la misma información, por lo tanto, intentaré combinar los eventos con el objetivo de encontrar **nuevos features**. 

Para comenzar divido en distintos DataFrames los distintos eventos, eliminando información que no es útil. 

In [4]:
viewedProduct  = tf[tf['event'] == 'viewed product'].loc[:,['hour','month','day','person','sku','model','condition','storage','color']]
brandListing = tf[tf['event'] == 'brand listing'].loc[:,['hour','month','day','person','skus']]
visitedSite = tf[tf['event'] == 'visited site'].loc[:,['hour','month','day','person','channel','new_vs_returning','city','region','country','device_type',\
                                                       'screen resolution','operating system version','browser version']]
adCampaignHit = tf[tf['event'] == 'ad campaign hit'].loc[:,['hour','month','day','person','url','campaign_source']]
genericListing = tf[tf['event'] == 'generic listing'].loc[:,['hour','month','day','person','skus']]
searchedProduct = tf[tf['event'] == 'searched products'].loc[:,['hour','month','day','person','skus','search_term']]
searchEngineHit = tf[tf['event'] == 'search engine hit'].loc[:,['hour','month','day','person','search engine']]
checkout = tf[tf['event'] == 'checkout'].loc[:,['hour','month','day','person','sku','color','storage','model','condition']]
staticPage = tf[tf['event'] == 'staticpage'].loc[:,['hour','month','day','person','static page']]
conversion = tf[tf['event'] == 'conversion'].loc[:,['hour','month','day','person','sku','model','color','condition','storage']]
lead = tf[tf['event'] == 'lead'].loc[:,['hour','month','day','person','model']]

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


**Buscar atributos de otro evento que no sea conversion**


La primera instancia que analizaré es si el usuario es nuevo o no, analizando del visited site el atributo 'new_vs_returning'

Intentaré determinar ahora si el usuario que realizó una conversión sólo realizó una única visita al sitio. Para ello debo fijarme en visited site si existe algún valor que sea returning, en tal caso, deja de ser nuevo. 

In [41]:
newVsReturning = visitedSite.loc[:,['person','new_vs_returning']]
newVsReturning = newVsReturning.groupby('person').new_vs_returning.value_counts().to_frame('cantidad')
newVsReturning = newVsReturning.unstack().fillna(0)
newVsReturning = newVsReturning.reset_index()
newVsReturning.columns = ['person','new','returning']
newVsReturning['esNuevo'] = newVsReturning['returning'] == 0

Agrego este atributo a conversion con un merge

In [42]:
labels_f = pd.merge(labels, newVsReturning.loc[:,['esNuevo','person']], how = 'left', on = 'person')

In [43]:
labels_f['esNuevo'].isnull().value_counts()

False    19126
True       288
Name: esNuevo, dtype: int64

Es decir que tengo 288 labels en los que no se si es nuevo o si ya volvió

Analizaré ahora la frecuencia de entradas al sitio de un determinado usuario. Para ello analizaré la relación entre new y returnings (le sumo 1 a la cantidad de returnings). 

In [44]:
newVsReturning['cantidadDeVisitas'] = (newVsReturning['new'] + newVsReturning['returning']).astype('int32')
labels_f = pd.merge(labels_f, newVsReturning.loc[:,['cantidadDeVisitas','person']],how = 'left', on = 'person')

In [45]:
labels_f['cantidadDeVisitas'].isnull().value_counts()

False    19126
True       288
Name: cantidadDeVisitas, dtype: int64

Me vuelve a dar lo mismo que antes, los resultados son consistentes. 

**Por lo tanto, el atributo esNuevo y cantidadDeVisitas no pueden aplicarse a 288 labels de 19414**

Analizaré si el usario ingresó a la página desde una publicidad (en algún momento) para ello usaré el evento ad_campaing_hit.

In [46]:
personasAdHit = adCampaignHit.loc[:,['person']]
personasAdHit['ingresoPorPublicidad'] = True
personasAdHit = personasAdHit.drop_duplicates()

In [47]:
labels_f = pd.merge(labels_f, personasAdHit, on = 'person', how = 'left')
labels_f = labels_f.fillna(False)

In [48]:
labels_f['ingresoPorPublicidad'].isnull().value_counts()

False    19414
Name: ingresoPorPublicidad, dtype: int64

Supongo que la información que no tengo en ad_campaing_hit de los usuarios, es porque no ingresaron por una campaña publicitaria alguna vez a la página y por lo tanto **este atributo puede usarse para lo 19414 labels**. 

Analizaré por qué campaña ingresaron los usuarios (en algún momento y luego realizaron una conversión), agregando esa información de ad_campaing_hit

In [49]:
campaniaPersonasAdHit = adCampaignHit.loc[:,['person','campaign_source']]
campaniaPersonasAdHit = campaniaPersonasAdHit.groupby('person').campaign_source.value_counts().to_frame('cantidad')
campaniaPersonasAdHit = campaniaPersonasAdHit.unstack().fillna(0).reset_index()
campaniaPersonasAdHit.head()
campaniaPersonasAdHit.columns=['person','cant_facebook','cant_facebookAds', 'cant_facebookSocial', 'cant_marketing_social', 'cant_afiliado', 'cant_afilio', 'cant_bing', 'cant_blog','cant_buscape', 'cant_criteo', 'cant_datacrush', 'cant_emblue', 'cant_gizmodo', 'cant_google', 'cant_indexa', 'cant_manifest', 'cant_mercadopago', 'cant_onsite', 'cant_rakuten','cant_rtbhouse', 'cant_voxus', 'cant_yotpo', 'cant_zanox']

**FALTA COMPLETAR ESTO, SON MUCHAS CAMPAÑAS**

El único evento que posee información sobre el dispositivo al cual se conecto es Visited Site. Se registra un Visited Site cada vez que el usuario ingresa a Trocafone (cualquiera sea el motivo). Repetiré el análisis hecho en el análisis exploratorio en el que mostrábamos que la mayoría de los usuarios se conectaban por un sólo dispositivo.


In [50]:
def contar_columnas(x):#(col1,col2,col3,col4):
    t=0
    if x['tablet']>0:
        t += 1
    if x['smartphone']>0:
        t += 1
    if x['computer']>0:
        t += 1
    if x['unknown']>0:
        t += 1
    return t


#print(visited_site.device_type.value_counts()) -> LOS UNKNOWN SON DESPRECIABLES. 
dispositivos= visitedSite.groupby('person')['device_type'].value_counts().to_frame('cantidad')
dispositivos = dispositivos.unstack().fillna(0).reset_index()
dispositivos.columns = ['person','computer','smartphone','tablet','unknown']
dispositivos = dispositivos.set_index('person')
dispositivos['porcentaje'] = dispositivos.apply(lambda p:contar_columnas(p), axis = 1)


print(dispositivos['porcentaje'].value_counts(normalize = True).to_frame())

#quito aquellos que se hayan conectado por más de un dispositivo
dispositivos = dispositivos[dispositivos['porcentaje'] <= 1]
visited_device = visitedSite.loc[:,['person','device_type']].drop_duplicates()
dispositivos = dispositivos.reset_index()
visited_device = pd.merge(dispositivos.loc[:,['person']],visited_device, on= 'person',how = 'left').drop_duplicates()


   porcentaje
1    0.959547
2    0.039721
3    0.000732


Por lo tanto existe un 96 porciento de los usuarios que se conectaron de un único tipo de dispositivo, por lo tanto, despreciaremos el otro 4 porciento. 


In [51]:
labels_f = pd.merge(labels_f, visited_device, how = 'left', on = 'person')
#le cambio el nombre a la columna device type para que no se confunda con lo que compraron

In [52]:
labels_f.device_type.isnull().value_counts()

False    18370
True      1044
Name: device_type, dtype: int64

In [53]:
labels_f['deviceSmartphone'] = labels_f['device_type'] == 'smartphone'
labels_f['deviceComputer'] = labels_f['device_type'] == 'computer'
labels_f['deviceTablet'] = labels_f['device_type'] == 'tablet'
labels_f['deviceUnknown'] = labels_f['device_type'] == 'unknown'

Es decir que **de los 19414 labels el atributo device_type presenta información para 18370** entonces 1044 labels no aportan este tipo de información. 

Analizaré ahora la cantidad de veces que el usuario vio un producto 

In [54]:
cantidadDeVisitasAProductos = viewedProduct.loc[:,['person']]
cantidadDeVisitasAProductos['cantVisitasAVP'] = 1
cantidadDeVisitasAProductos = cantidadDeVisitasAProductos.groupby('person').cantVisitasAVP.sum().to_frame()


In [55]:
labels_f = pd.merge(labels_f, cantidadDeVisitasAProductos, on= 'person', how = 'left')

In [56]:
labels_f.cantVisitasAVP.isnull().value_counts()

False    18570
True       844
Name: cantVisitasAVP, dtype: int64

Supongo que como no tenemos información de 844 labels sobre la cantidad de visitas a productos estos nunca lo hicieron.

In [57]:
personasSinVisitas = labels_f.loc[:,['person','cantVisitasAVP']]
personasSinVisitas = personasSinVisitas.fillna(0)

In [58]:
labels_f = pd.merge(labels_f, personasSinVisitas, how = 'left', on = 'person')

In [59]:
labels_f = labels_f.loc[:,['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP_y']]
labels_f.columns = ['person','label','esNuevo','cantidadDeVisitas','ingresoPorPublicidad','deviceSmartphone','deviceComputer','deviceTablet','deviceUnknown','cantVisitasAVP']

Usando visitedSite analizaré el momento del día en el que el usuario realizó las visitas. Lo agruparé en las siguientes clases:

-Mañana: si el usuario realizó la visita entre las 7 y 12 hs. 

-Tarde: si el usuario realizó la visita entre las 12 y las 18 hs. 

-Noche: si el usuario realizó la visita entre las 18 y 24 hs. 

-Madrugada: si el usuario realizó la visita entre las 24 y 7 hs. 

In [60]:
momentoDeVisita = visitedSite.loc[:,['person','hour']]
momentoDeVisita['maniana'] = ((momentoDeVisita['hour'] < 12 ) & (momentoDeVisita['hour'] >= 7)).astype('int32')
momentoDeVisita['tarde'] = ((momentoDeVisita['hour'] < 18 ) & (momentoDeVisita['hour'] >= 12)).astype('int32')
momentoDeVisita['noche'] = ((momentoDeVisita['hour'] < 24 ) & (momentoDeVisita['hour'] >= 18)).astype('int32')
momentoDeVisita['madrugada'] = ((momentoDeVisita['hour'] < 7 ) & (momentoDeVisita['hour'] >= 24)).astype('int32')
momentoDeVisita = momentoDeVisita.groupby('person').sum().reset_index()
momentoDeVisita = momentoDeVisita.loc[:,['person','maniana','tarde','noche','madrugada']]

In [61]:
labels_f = pd.merge(labels_f,momentoDeVisita, on = 'person', how = 'left') 

In [62]:
labels_f['maniana'].isnull().value_counts()

False    19126
True       288
Name: maniana, dtype: int64

Por lo tanto, **los atributos: maniana, tarde, noche y madrugada no fueron asignados para 288 labels**. Esto concuerda con el atributo device type obtenido anterior mentde del mismo evento: visited site.

Analizaré el canal por el que el usuario realiza la visita

In [63]:
canalDeVisita = visitedSite.loc[:,['person','channel']]
canalDeVisita = canalDeVisita.groupby('person').channel.value_counts().to_frame('cantidad')
canalDeVisita = canalDeVisita.unstack()
canalDeVisita = canalDeVisita.reset_index()
canalDeVisita.columns = ['person','cantDirect','cantEmail','cantOrganic','cantPaid','cantReferral',\
                           'cantSocial','cantUnknown']
canalDeVisita = canalDeVisita.fillna(0)

In [64]:
labels_f = pd.merge(labels_f,canalDeVisita,on = 'person', how = 'left')

In [65]:
labels_f['cantDirect'].isnull().value_counts()

False    19126
True       288
Name: cantDirect, dtype: int64

In [66]:
labels_f.columns

Index(['person', 'label', 'esNuevo', 'cantidadDeVisitas',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown'],
      dtype='object')

In [67]:
labels_f = labels_f.dropna()
labels_f

,person,label,esNuevo,cantidadDeVisitas,ingresoPorPublicidad,deviceSmartphone,deviceComputer,deviceTablet,deviceUnknown,cantVisitasAVP,...,tarde,noche,madrugada,cantDirect,cantEmail,cantOrganic,cantPaid,cantReferral,cantSocial,cantUnknown
0,0566e9c1,0,False,17,True,False,False,False,False,23.0,...,7.0,5.0,0.0,8.0,0.0,1.0,0.0,8.0,0.0,0.0
1,6ec7ee77,0,True,1,True,False,False,False,False,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,abe7a2fb,0,False,22,True,False,False,False,False,31.0,...,7.0,7.0,0.0,2.0,0.0,2.0,6.0,10.0,2.0,0.0
3,34728364,0,False,4,False,False,False,False,False,24.0,...,0.0,2.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0
4,87ed62de,0,True,1,True,False,False,False,False,9.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,db2c4d27,1,False,114,True,False,False,False,False,121.0,...,57.0,35.0,0.0,24.0,0.0,57.0,1.0,32.0,0.0,0.0
6,cde431db,0,False,3,True,False,False,False,False,8.0,...,2.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0
7,be65035b,0,False,3,False,False,False,False,False,0.0,...,0.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0
8,a4178891,0,True,1,True,False,False,False,False,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,d066f64c,0,True,1,True,False,False,False,False,12.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Por lo tanto, **los atributos: cantDirect, cantEmail, cantOrganic, cantPaid, cant referral, cantSocial, cantUnknown que refieren a la cantidad de accesos hecho por cada canal, no tiene fueron asignados para 288 labels**. Esto concuerda con todos los atributos obteneidos del mismo evento: vistedSite

In [68]:
labels_f.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19126 entries, 0 to 19413
Data columns (total 21 columns):
person                  19126 non-null object
label                   19126 non-null int64
esNuevo                 19126 non-null bool
cantidadDeVisitas       19126 non-null object
ingresoPorPublicidad    19126 non-null bool
deviceSmartphone        19126 non-null bool
deviceComputer          19126 non-null bool
deviceTablet            19126 non-null bool
deviceUnknown           19126 non-null bool
cantVisitasAVP          19126 non-null float64
maniana                 19126 non-null float64
tarde                   19126 non-null float64
noche                   19126 non-null float64
madrugada               19126 non-null float64
cantDirect              19126 non-null float64
cantEmail               19126 non-null float64
cantOrganic             19126 non-null float64
cantPaid                19126 non-null float64
cantReferral            19126 non-null float64
cantSocial          

In [70]:
X = labels_f.loc[:,[ 'esNuevo',
       'ingresoPorPublicidad', 'deviceSmartphone', 'deviceComputer',
       'deviceTablet', 'deviceUnknown', 'cantVisitasAVP', 'maniana', 'tarde',
       'noche', 'madrugada', 'cantDirect', 'cantEmail', 'cantOrganic',
       'cantPaid', 'cantReferral', 'cantSocial', 'cantUnknown']]
#saco pq no lo puedo usar en xgboost pq esta mal el tipo'cantidadDeVisitas'
y =labels_f.loc[:,'label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=7506)

In [71]:
#X = np.array(labels_f.ix[:, 1:1]) 
#y = np.array(labels_f['label']) 

clf = RandomForestClassifier()
clf.fit(X_train,y_train)

#pred = clf.predict(X_test)
#print(accuracy_score(y_test, pred))
preds = clf.predict(X_test)
train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9744960204496601
Test acuracy:  0.9477260846837429
ROC auc score:  0.49780340472267987
Confusion matrix: 
[[1813    8]
 [  92    0]]


In [72]:
data_dmatrix = xgb.DMatrix(data=X,label=y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

In [77]:
xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, scale_pos_weight = 1, min_child_weight=1)

In [78]:
xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [79]:
preds = xg_reg.predict(X_test)

In [80]:
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.951045751633987
Test acuracy:  0.9464192368008364
ROC auc score:  0.501797907016641
Confusion matrix: 
[[3620    5]
 [ 200    1]]
